<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_0306_MutiLN_mod25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mod 25 實驗結果測試
<a href="https://drive.google.com/file/d/1RzHAFqRE0sV5uXkwbzjXXo0uOHL8SFRo/view?usp=sharing">請見paper</a><br>
把每一層都做LN<br>
要對 G4 做正規化

In [ ]:
from google.colab import drive
drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [3]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense, Flatten, LayerNormalization, Input,Activation, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from random import choice, sample
import math


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(primeTable, limit, addSize=5):
    halfList = [num for num in primeTable if num <= int(limit//2)]
    if len(halfList) < 5:
      halfList += [0] * (5 - len(halfList))
    return sample(halfList, addSize)

def add_mod(primeTable25, num):
  return [num%index for index in primeTable25]

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


def compute_coprimes(n): ## 獲得質因數 zero padding 到40
    coprimes = []
    lenth = 50
    for num in range(1,n):
        if gcd(num , n) == 1:
            coprimes.append(num)
        if len(coprimes) >= lenth:
            return coprimes
    coprimes = coprimes + [0] * (lenth - len(coprimes))
    return coprimes


In [ ]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_G4_5000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (len(trainDataNumbers))

5000


In [ ]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [ ]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [ ]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []

for num in shuffledNumbers:

    #add 25 prefix prime
    feature = []
    feature = add_mod(primeTable25, num)
    mergedTrainNumber.append(feature)

for num in shuffledLabel:
    mergedTrainLabel.append(num)

    print("merge number ", num, " done.")

#print (mergedTrainNumber)

In [ ]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)
print ("X Train [0] value : \n",X_train[0])
print ("X Train [0] label : ",y_train[0])
print ("=======")
print ("X shpae : " , X_train[0].shape)

X Train [0] value : 
 [ 0  0  4  1  0  4 16  0 15 18  4 18 18 23 30  1  2 40 41 49 71 56 54 75
 60]
X Train [0] label :  3387.672185
X shpae :  (25,)


In [ ]:
# start training
# 清空所有
K.clear_session()

################################################################

def LN_Layer(x,u1,u2):
  # two group
  group1 = x
  group2 = Activation('relu')(x)
  group2 = Lambda(lambda x: K.log(x + 1))(group2)  # 使用 natural logarithm

  # G1 -> next
  group1 = Dense(units=u1, activation='relu')(group1)

  # G2 -> exp -> next
  group2 = Dense(units=u2, activation='linear')(group2)
  group2 = Activation('exponential')(group2)

  # concate
  merged = concatenate([group1, group2], axis=-1)

  return merged


# require gpu
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    inputLayer = Input(shape=X_train[0].shape)

    # LN_1
    LN_1 = LN_Layer(inputLayer,25,200)

    # LN
    LN_2 = LN_Layer(LN_1,120,80)

    # 後加兩層
    LN_3 = LN_Layer(LN_2,120,80)

    #Output
    outputLayer = Dense(units=1, activation='linear')(LN_3)
    model = Model(inputs=inputLayer , outputs=outputLayer)


    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.001)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer,
                  metrics=['accuracy'])    # train version 2

    # train model
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200,batch_size=2048, validation_data=(X_val, y_val))
    print("--------------------------------------------------------")

    # save model
    #model.save('./csv/goldbach_model_1112_400w.h5')
print("model saved.")


--- gpu /device:GPU:0 is loaded. ---
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25)]                 0         []                            
                                                                                                  
 activation (Activation)     (None, 25)                   0         ['input_1[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 25)                   0         ['activation[0][0]']          
                                                                                                  
 dense_1 (Dense)             (None, 200)                  5200      ['lambda[0][0]']              
                                                         

KeyboardInterrupt: 

## predict

In [1]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [4]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

---loading primes table---
[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [5]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/Drive/MyDrive/實驗/model_save/goldbach_model_0306.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/test_set_G4_7M.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values
testDataG4 = testData['G4'].values
testDataPartition = testData['Partition'].values

print("---loading test data done.---")


---loading test data done.---


In [6]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []

for num in testDataNumbers:

    ### mod25
    feature = []
    feature = add_mod(primeTable25, num)
    mergedTestNumber.append(feature)

for num in testDataPartition:
    mergedTestLabel.append(num)


    #print("merge number ", num, " is done.")

#print (mergedTrainNumber)

In [7]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [8]:
print (mergedTestNumber[0])
print (mergedLabel[0])

[0, 0, 4, 0, 3, 2, 14, 12, 4, 25, 27, 18, 0, 29, 37, 40, 31, 14, 55, 41, 28, 68, 65, 12, 68]
58853


In [9]:
evaluation = model.evaluate(mergedArray, mergedLabel)
#XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)



2/2 [==============================] - 0s 11ms/step - loss: 1433156864.0000 - accuracy: 0.0000e+00
Loss: 1433156864.0
Accuracy: 0.0
##########################
2/2 [==============================] - 0s 6ms/step


# +N 才要執行正規

In [10]:

print (f"將50筆測資正規化回G4")
normalizedTest = []
normalizedPred = []
for index in range(len(testDataLabel)):
  normalizedTest.append(testDataLabel[index]*testDataG4[index]) # normalize ground truth
  normalizedPred.append(predictions[index]*testDataG4[index])  # normalize predition


將50筆測資正規化回G4


In [11]:
print (f"將50筆測資正規化回G4")
for index, number in enumerate(testDataNumbers):
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")

將50筆測資正規化回G4
第0筆資料為數字:	7984914	|正確答案 = 58853	模型預估 = 59592
第1筆資料為數字:	7050792	|正確答案 = 51667	模型預估 = 51957
第2筆資料為數字:	7852768	|正確答案 = 31367	模型預估 = 31700
第3筆資料為數字:	7807448	|正確答案 = 26161	模型預估 = 26110
第4筆資料為數字:	7695230	|正確答案 = 31001	模型預估 = 31079
第5筆資料為數字:	7109000	|正確答案 = 28859	模型預估 = 29023
第6筆資料為數字:	7268430	|正確答案 = 64326	模型預估 = 64694
第7筆資料為數字:	7147384	|正確答案 = 22306	模型預估 = 22510
第8筆資料為數字:	7555772	|正確答案 = 27371	模型預估 = 27439
第9筆資料為數字:	7066874	|正確答案 = 21654	模型預估 = 21690
第10筆資料為數字:	7632520	|正確答案 = 36879	模型預估 = 37245
第11筆資料為數字:	7039812	|正確答案 = 46596	模型預估 = 47057
第12筆資料為數字:	7670862	|正確答案 = 46176	模型預估 = 46590
第13筆資料為數字:	7402232	|正確答案 = 22333	模型預估 = 22477
第14筆資料為數字:	7432508	|正確答案 = 22866	模型預估 = 23020
第15筆資料為數字:	7230862	|正確答案 = 22140	模型預估 = 22186
第16筆資料為數字:	7305346	|正確答案 = 22305	模型預估 = 22424
第17筆資料為數字:	7349124	|正確答案 = 47254	模型預估 = 47632
第18筆資料為數字:	7134356	|正確答案 = 22965	模型預估 = 23285
第19筆資料為數字:	7990454	|正確答案 = 23942	模型預估 = 24094
第20筆資料為數字:	7024954	|正確答案 = 22002	模型預估 = 21858
第21筆資料為數字:	7715734	|正確答案 = 2551

<ipython-input-11-0f6ad44a8f98>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案 = {testDataPartition[index]:05d}\t模型預估 = {int(normalizedPred[index])}")


## 把所有資料zip

In [ ]:
gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]


normalizedPred_int = [int(arr) for arr in normalizedPred]
combined_data = list(zip(testDataNumbers, testDataPartition, normalizedPred_int, gap))
sort_data = sorted(combined_data, key = lambda x : x[0])


print (f"test on {testDataPath[39::]}")
print (f"測試數字/正確答案/預測答案/差距量\n")
for i in sort_data:
  print (i)

print ("***")
fourth_items = [item[0] for item in sort_data]
for i in fourth_items:
  print (i)
# print(fourth_items)

test on test_set_G4_7M.csv
測試數字/正確答案/預測答案/差距量

(7000034, 21393, 21412, 19)
(7024954, 22002, 21858, 143)
(7037344, 21553, 21684, 131)
(7039812, 46596, 47057, 461)
(7049660, 28538, 28826, 288)
(7050792, 51667, 51957, 290)
(7066874, 21654, 21690, 36)
(7097916, 51956, 52383, 427)
(7109000, 28859, 29023, 164)
(7121414, 22558, 22609, 51)
(7134356, 22965, 23285, 320)
(7147384, 22306, 22510, 204)
(7166374, 21786, 22036, 250)
(7177008, 43561, 43992, 431)
(7229336, 22227, 22438, 211)
(7230862, 22140, 22186, 46)
(7236930, 59226, 59277, 51)
(7243074, 44362, 44418, 56)
(7259704, 23166, 23452, 286)
(7268224, 22013, 22319, 306)
(7268430, 64326, 64694, 368)
(7305346, 22305, 22424, 119)
(7349124, 47254, 47632, 378)
(7389732, 53823, 54117, 294)
(7399392, 65325, 65446, 121)
(7402232, 22333, 22477, 144)
(7432508, 22866, 23020, 154)
(7472958, 45171, 45564, 393)
(7502200, 30025, 30602, 577)
(7555772, 27371, 27439, 68)
(7576878, 46031, 46177, 146)
(7606482, 50630, 50313, 316)
(7632520, 36879, 37245, 366)
(76

<ipython-input-240-f3f6e207658d>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gap = [abs(int(partition - pred)) for partition, pred in zip(testDataPartition, normalizedPred)]
<ipython-input-240-f3f6e207658d>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  normalizedPred_int = [int(arr) for arr in normalizedPred]


## RMSE and error rate

In [ ]:
def calculate_mse(predictions, targets):
    """
    計算均方誤差 (MSE)

    參數:
    predictions: 預測值的列表或陣列
    targets: 實際值的列表或陣列，與預測值對應

    返回值:
    mse: 均方誤差
    """
    # 確保預測值和實際值的長度相等
    if len(predictions) != len(targets):
        raise ValueError("預測值和實際值的長度不一致")

    # 計算平方誤差
    squared_errors = [(p - t) ** 2 for p, t in zip(predictions, targets)]

    # 計算均方誤差
    mse = sum(squared_errors) / len(predictions)

    return mse

# 使用 LN
mse_result = calculate_mse(normalizedPred, testDataPartition)

# 使用 MLP(old)
#mse_result = calculate_mse(predictions, testDataPartition)

mean = testData['Partition'].mean()

#LN MSE
MSE = int(calculate_mse(normalizedPred, testDataPartition))

#MLP MSE
# MSE = int(calculate_mse(predictions, testDataPartition))

RMSE= math.sqrt(MSE)
errorRate = RMSE/mean

print (f"MSE : {MSE}\nRMSE : {RMSE}\nError Rate : {errorRate}  ->  {round(errorRate,7)*100}%")

均方誤差 (MSE): [82897.31]
